# 6.1 Classification Models

![blr](../../images/blr.jpeg)

## 6.1.1 Binary Models. Churn Example

Al hablar de modelos de clasificación binarios estamos hablando de modelos en los que nuestra variable dependiente es categórica y únicamente tiene dos clases a predecir, uno de los ejemplos más claros de este tipo de modelos son los problemas de 'Churn' o lo que es lo mismo la retención de clientes por parte de las compañías, y más que la retención es la detección temprana de si un cliente se va a ir de nuestra compañía o no. Suelen ser problemas complejos ya que la clase a predecir suele estar muy desbalanceada, ya que lo normal es que los clientes no se vayan de la compañía.

![churn](../../images/churn.png)


![churn2](../../images/churn2.png)


### **Regresión Logística**

In [ ]:
import pandas as pd                                # panel data, for handling dataframes
pd.set_option('display.max_columns', None)         # show all columns of the dataframe

import numpy as np                                 # numerical python, linear algebra library

import pylab as plt                                # plotting library
import seaborn as sns                              # plotting library
sns.set(style='white')                             # seaborn style


from sklearn.linear_model import LogisticRegression            # logistic regression model   

from sklearn.preprocessing import MinMaxScaler                 # standarized
from sklearn.preprocessing import LabelEncoder                 # Para codificar nuestra variable a predecir

from sklearn.model_selection import train_test_split     # split data into train and test sets

import warnings
warnings.filterwarnings('ignore')

Comenzaremos por explicar el modelo de regresión logisitica, modelo básico utilizado para clasificar, se basa en minimizar los pesos de la función sigmoide.

![sig_plot](../../images/sig_plot.png)

Básicamente la función tratará de ajustarse para separar nuestros datos en dos grupos los que se encuentren a la izq de la función perteneceran a una clase, y los de la derecha a la otra.

#### Carga de datos

In [ ]:
df=pd.read_csv('../../../data/churn.csv')

df.head()

#### EDA

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.Churn.value_counts()

In [ ]:
df.ChurnBinary.value_counts()

In [ ]:
(df.Churn.value_counts()/len(df)).plot.bar(color=['g', 'r'],    # plot customer churn rate
                                           figsize=(10, 6),
                                           title='Churn Rate',
                                           rot=0,
                                           fontsize=12);

In [ ]:
df.Churn.value_counts(normalize=True)

#### Transformaciones

Tenemos varias opciones, podríamos tratar de crear un modelo únicamente con las variables numéricas que tenemos y ver cual es su desempeño, sería el modelo más simple y el que menos coste tendría, podríamos proponer alguna serie de transformaciones de las columnas categóricas y numéricas para poder realizar varios experimentos, pero para el caso que nos compete, realizaremos una transformación con label Enconder.

##### Variables categóricas

**Label Encoder**

In [ ]:
df.PhoneService.value_counts()

In [ ]:
dicto = {'Yes':1, 'No':0}

prueba = df.PhoneService.apply(lambda x: dicto[x])
prueba

In [ ]:
le = LabelEncoder()

In [ ]:
df_le = df.copy()
for c in df_le.columns:
    
    if df_le.dtypes[c]==object and (c != 'customerID' or c != 'Churn'):
        
        le.fit(df_le[c].astype(str))
        
        df_le[c]=le.transform(df_le[c].astype(str))

In [ ]:
df_le.head()

In [ ]:
df.head()

In [ ]:
df.MultipleLines.value_counts()

In [ ]:
df_le.MultipleLines.value_counts()

##### Variables numéricas

Al igual que nos pasaba con las variables categóricas debemos de decidir que hacemos con las variables numéricas, debemos ver si estandarizamos los datos o no, para ello nos fijaremos en la descripción de nuestros datos estadísticos.

In [ ]:
df.describe().T

In [ ]:
df_le.describe().T

Como la mayoría de nuestros datos están entre 0 y 1 podemos usar el MinMaxScaler para transformar las columnas.

**Antes de aplicar las transformaciones realizamos el train test split**

Al igual que en los modelos de regresión antes de realizar las transformaciones para estandarizar los datos haremos el train test split. En el caso de que tuvieramos varios experimentos deberíamos de hacerlel tts para cada uno de nuestros dataframes propuestos para realizar los experimentos. Y para cada uno de ellos deberíamos de entrenar un escaler diferente.

In [ ]:
X_train_le, X_test_le, y_train_le, y_test_le = train_test_split(df_le.drop(['customerID', 'Churn', 'ChurnBinary'], axis=1), df_le.Churn, random_state=42, test_size=.2, 
                                                    stratify=df_le.Churn)

In [ ]:
y_train_le.value_counts(), y_test_le.value_counts()

In [ ]:
mm_le = MinMaxScaler().fit(X_train_le)


In [ ]:
X_train_le = mm_le.transform(X_train_le)
X_test_le = mm_le.transform(X_test_le)

In [ ]:
pd.DataFrame(X_train_le).describe().T

In [ ]:
y_train_le.value_counts(normalize=True), y_test_le.value_counts(normalize=True)

#### Modelo

Lo primero que tenemos que hacer es inicializar nuestro modelo

In [ ]:
logreg_le = LogisticRegression(max_iter=2000)

In [ ]:
logreg_le.fit(X_train_le, y_train_le)
pred = logreg_le.predict(X_train_le)
score_train = logreg_le.score(X_train_le, y_train_le) #predicciones y evaluación sobre train
score_test = logreg_le.score(X_test_le, y_test_le) #predicciones y evaluación sobre test
        
res_num = {'le_train_score': score_train,
                   'le_test_score': score_test}

In [ ]:
res_num

Parece ser que tenemos un buen resultado pero el accuracy en modelos de clasificación no es la mejor métrica de evaluación

##### Evaluación

**Matriz de confusión**

![conf_matrix](../../images/conf_matrix.jpeg)

+ TP := True Positive (aciertos clase 1)
+ TN := True Negative (aciertos clase 0)
+ FP := False Positive (Error tipo I, decir 1 cuando es 0)
+ FN := False Negative (Error tipo II, decir 0 cuando es 1)

+ Accuracy  := (TP+TN)/(TP+TN+FP+FN) (acierto)  ($\frac{1}{n}\sum 1(\hat{y_i}=y_i$))
+ Precision := TP/(TP+FP)
+ Recall    := TP/(TP+FN)  (Sensibilidad, TPR)
+ F1_Score  := 2·Recall·Precision/(Recall+Precision)

(F1 funciona mejor que el accuracy cuando los datos no están balanceados y cuando FP y FN son muy diferentes)

![f1](../../images/f1.png)

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, recall_score, precision_score

In [ ]:
logreg_le.fit(X_train_le, y_train_le)
score_train = logreg_le.score(X_train_le, y_train_le)
score_test = logreg_le.score(X_test_le, y_test_le)
precision_train = precision_score(y_train_le, logreg_le.predict(X_train_le))
precision_test = precision_score(y_test_le, logreg_le.predict(X_test_le))
recall_train = recall_score(y_train_le, logreg_le.predict(X_train_le))
recall_test = recall_score(y_test_le, logreg_le.predict(X_test_le))
f1_train = f1_score(y_train_le, logreg_le.predict(X_train_le))
f1_test = f1_score(y_test_le, logreg_le.predict(X_test_le))
        
res_num = {'le_train_score': score_train,
           'le_test_score': score_test,
           'le_train_precision': precision_train,
           'le_test_precision': precision_test,
           'le_train_recall': recall_train,
           'le_test_recall': recall_test,
           'le_f1_train': f1_train,
           'le_f1_test': f1_test}
        
sns.heatmap(confusion_matrix(y_train_le, logreg_le.predict(X_train_le)), annot=True)
plt.title('Confusion Matrix Train')
plt.show();
sns.heatmap(confusion_matrix(y_test_le, logreg_le.predict(X_test_le)), annot=True)
plt.title('Confusion Matrix Test')
plt.show();

In [ ]:
res_num

Como vemos nuestro modelo no está funcionando muy correctamente, y esto es debido a que nuestros datos están muy desbalanceados, y nuestro modelo tiene un sesgo, básicamente está diciendo a todo que no, a parte está ligeramente overfitteado, esto podemos verlo en los resultados de train y test, ya que el f1 score en test es ligeramente superior que en train.
Para tratar de compensar este tipo de incovenientes tenemos diferentes técnicas de balanceo

#### Balanceo

In [ ]:
sns.countplot(df.Churn);

Cuando la variable objetivo está desbalanceada, tenemos varias opciones para tratar de balancear los datos:

    - Métodos de undersampling
    - Métodos de oversampling
 
Elegiremos uno u otro método en base al número de datos de los que dispongamos, si disponemos un gran volumen de datos podemos decantarnos por técnicas de undersamplig en las que básicamente nos quedaremos con el mayor número de datos de la clase minoritaria, y tomaremos un sample de la misma longitud de la clase mayoritaria.

Para los métodos de oversampling tenemos dos métodos:

    - Random Oversampling: Con esta técnica el algoritmo genera datos nuevos de forma aleatoria basándose en los datos que le pasamos, hasta igualar ambas clases.
    
    - Smote: Este método es similar con la salvedad de que los datos generados de forma aleatoria se generan siguiendo una distribución normal.

In [ ]:
#!pip install -U imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE, RandomOverSampler

In [ ]:
smote = SMOTE()
rov = RandomOverSampler()

**Es importante tener en cuenta que solo aplicaremos la técnica de Oversampling o Undersampling a los datos de entrenamiento, los datos de test no se tocan.**

In [ ]:
X_train_le_sm, y_train_le_sm = smote.fit_resample(X_train_le, y_train_le)


In [ ]:
X_train_le_sm.shape, y_train_le_sm.shape

In [ ]:
y_train_le_sm.value_counts()

In [ ]:
sns.histplot(y_train_le_sm);

In [ ]:
logreg_le.fit(X_train_le_sm, y_train_le_sm)
score_train_sm = logreg_le.score(X_train_le_sm, y_train_le_sm)
score_test_sm = logreg_le.score(X_test_le, y_test_le)
precision_train_sm = precision_score(y_train_le_sm, logreg_le.predict(X_train_le_sm))
precision_test_sm = precision_score(y_test_le, logreg_le.predict(X_test_le))
recall_train_sm = recall_score(y_train_le_sm, logreg_le.predict(X_train_le_sm))
recall_test_sm = recall_score(y_test_le, logreg_le.predict(X_test_le))
f1_train_sm = f1_score(y_train_le_sm, logreg_le.predict(X_train_le_sm))
f1_test_sm = f1_score(y_test_le, logreg_le.predict(X_test_le))
        
res_sm = {'le_train_sm_score': score_train_sm,
          'le_test_sm_score': score_test_sm,
          'le_train_precision': precision_train_sm,
          'le_test_precision': precision_test_sm,
          'le_train_recall': recall_train_sm,
          'le_test_recall': recall_test_sm,
          'le_f1_train_sm': f1_train_sm,
          'le_f1_test_sm': f1_test_sm}
        
sns.heatmap(confusion_matrix(y_train_le_sm, logreg_le.predict(X_train_le_sm)), annot=True);
plt.title('Confusion Matrix Train')
plt.show();
sns.heatmap(confusion_matrix(y_test_le, logreg_le.predict(X_test_le)), annot=True);
plt.title('Confusion Matrix Test')
plt.show();

In [ ]:
res_sm

#### Selección de Variables y reentreno de modelo

Como podemos oberservar el modelo ha mejorado un poco y se ha corregido el overfitting, llegados a este punto podemos valorar varias opciones o bien realizar un análisis más profundo de los datos y ver como están correlacionados nuestros datos en busca de colinealidad, realizar un estudio estudio de importancia de características, ajustar punto donde queremos valorar que un resultado sea Churn en la regresión logística, o cambiar de modelo.
Antes de cambiar de modelo vamos a probar a ajustar el punto de intersección de la regresión logística y evaluar los coeficientes de cada una de las características y su correlación en busca de colinealidad.

In [ ]:
logreg_le.intercept_

In [ ]:
coefs = dict(zip(list(df_le.drop(['customerID', 'Churn', 'ChurnBinary'], axis=1).columns),list(logreg_le.coef_[0])))

In [ ]:
coefs

Vamos a interpretar estos coeficientes, también denominados R statistic.

Un valor positivo significa que al crecer la variable predictora, lo hace la probabilidad de que el evento ocurra. Un valor negativo implica que si la variable predictora decrece, la probabilidad de que el resultado ocurra disminuye. Si una variable tiene un valor pequeño de R entonces esta contribuye al modelo sólo una pequeña cantidad.

De esto podemos extraer que si quitamos todas las columnas con un coeficiente negativo, nuestro modelo podría mejorar.

In [ ]:
neg_coef = []

for k,v in coefs.items():
    if v < 0:
        neg_coef.append(k)
neg_coef

In [ ]:
df_le_pos_coef = df_le.drop(neg_coef+['customerID','ChurnBinary'], axis=1)

df_le_pos_coef.head()

In [ ]:
df_le_pos_coef.info()

También comprobaremos la correlación de nuestas columnas restantes en busca de posible colinealidad entre ellas

In [ ]:
def print_heatmap_corr(data:pd.DataFrame, annot:bool=True, cmap:str=None, 
                       mask:bool=True, save:bool=False, title:str=None)->None:
    
    '''
        Función que recibe un dataframe y devuelve la matriz de correlación en forma de mapa de color
        
        Parameters:
        -----------
        
        data: Dataset sobre el que queremos realizar la matriz de correlación
        annot: Si queremos mostrar el valor de la correlación en la matriz, default = True
        cmap: Paleta de colores que queremos usar para nuestro heatmap
        mask: Parámetro para mostrar solo la triangular inferior de la matriz de correlación
        save: Parámetro para salvar nuestro gráfico
        title: Título que queremos que lleve nuestro gráfico
    '''
    
    sns.set(style='white')     # estilo blanco hace que el fondo de la matriz sea transparente

    if mask: # Si mask es True
        mascara=np.triu(np.ones_like(data.corr(), dtype=bool))   # genera una mascara para tapar valores
    else:
        mascara = None # No aplicamos máscar

    if cmap: # Si le hemos pasado una paleta de colores
        c_map = sns.color_palette(cmap, as_cmap=True)
    else:
        c_map=sns.diverging_palette(0, 10, as_cmap=True)   # paleta de colores por defecto

    plt.figure(figsize=(20,15))
    p = sns.heatmap(data.corr(), # aplica el método corr() a nuestro dataset
            mask=mascara, # aplica la mascara
            cmap=c_map, # aplica la paleta de colores
            vmax=1, # para establecer el valor máximo de valores
            center=0, # establece el centro de la paleta de colores
            square=True,
            linewidth=0.5, # para aplicar borde a los cuadros de la matriz
            cbar_kws={'shrink': 0.5}, # mostrar leyenda de colores
            annot=annot # mostrar valores de la matriz
           )
    p.set_title(title, fontsize=20)
    
    if save:
        try:
            plt.savefig(f'graphics/{title}.png')
        except:
            destino = input('No exite la carpeta de destino, introduce un nombre para la carpeta de destino: ')
            os.mkdir(destino)
            plt.savefig(f'{destino}/{title}.png')
    
    plt.show();

In [ ]:
print_heatmap_corr(df_le_pos_coef)

Vemos que la las variables independientes no tienen mucha correlación con nuestra variable dependiente, esto quiere decir que la solución al problema es compleja y hay que tratarla con cuidado, y también puede darnos una indicación de que los resultados que podemos esperar de los modelos no van ha ser muy buenos, pero tenemos que tratar de hacer todo lo posible para que estos sean lo más altos posibles.
Respecto a la correlación entre las variables independientes vemos que salvo TotalCharges no hay excesiva colinealidad entre nuestras variables, por lo que nos quedaremos con ellas.

Como nuestro set de datos es diferente al original debemos de volver a realizar el train_test_split de nuevo, junto con el scaler y el smote para corregir el balanceo.

Declararemos de nuevo nuestras X e y y aplicamos todo el proceso de transformaciones.

In [ ]:
X = df_le_pos_coef.drop('Churn', axis=1)
y = df_le_pos_coef.Churn

X.shape, y.shape

**Primero hacemos train test split de nuevo**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=.2, stratify=y)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

**Seguidamente aplicamos el escalado**

In [ ]:
mm = MinMaxScaler().fit(X_train)

In [ ]:
X_train = mm.transform(X_train)
X_test = mm.transform(X_test)

**Aplicamos SMOTE**

In [ ]:
smote = SMOTE()

In [ ]:
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
y_train.value_counts()

**Inicializamos un modelo nuevo y lo entrenamos**

In [ ]:
lr = LogisticRegression(max_iter=2000)

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
score_train = lr.score(X_train, y_train)
score_test = lr.score(X_test, y_test)
precision_train = precision_score(y_train, lr.predict(X_train))
precision_test = precision_score(y_test, lr.predict(X_test))
recall_train = recall_score(y_train, lr.predict(X_train))
recall_test = recall_score(y_test, lr.predict(X_test))
f1_train = f1_score(y_train, lr.predict(X_train))
f1_test = f1_score(y_test, lr.predict(X_test))
        
res = {'lr_train_score': score_train,
       'lr_test_score': score_test,
       'lr_train_precision': precision_train,
       'lr_test_precision': precision_test,
       'lr_train_recall': recall_train,
       'lr_test_recall': recall_test,
       'lr_f1_train': f1_train,
       'lr_f1_test': f1_test}
        
sns.heatmap(confusion_matrix(y_train, lr.predict(X_train)), annot=True);
plt.title('Confusion Matrix Train')
plt.show();
sns.heatmap(confusion_matrix(y_test, lr.predict(X_test)), annot=True);
plt.title('Confusion Matrix Test')
plt.show();

In [ ]:
res

##### Ajuste del umbral de probabilidad

Vemos que con este cambio hemos corregido un poco el tema del overfit pero los resultados del modelo en test siguen sin ser muy buenos, por lo que como último recurso antes de descartar el modelo y probar con otro vamos a tratar de ajustar el umbral de nuestra regresión a ver si con ello podemos mejorar sus f1_score, para ello en vez de usar el método predict, usaremos el método predict_proba, que lo que hará será devolvernos una lista con la probabilidad de que ocurra el evento de cada una de nuestras clases predictoras.

In [ ]:
probas = lr.predict_proba(X_train)
probas[:3]

Vamos a tratar de buscar un umbral óptimo para nuestra regresión, si tenemos en cuenta que un 26% de nuestros datos pertenecen a la clase 1 , es decir, Churn (clientes que se van), vamos a bajar el umbral en la clase para que de está manera el modelo no diga siempre que no a todo.

In [ ]:
preds = [1 if probas[i][1]>0.30 else 0 for i in range(len(probas))]
preds[:10]

Ya tenemos nuestras predicciones en base a nuestro nuevo umbral, ahora vamos a comprobar si nuestro modelo ha mejorado

In [ ]:
recall_proba = recall_score(y_train, preds)
precision_proba = precision_score(y_train, preds)
f1_score_proba = f1_score(y_train, preds)
print(f'Recall: {recall_proba},\nPrecision: {precision_proba},\nf1: {f1_score_proba}')

In [ ]:
sns.heatmap(confusion_matrix(y_train, preds), annot=True)
plt.title('Confusion Matrix Probas Train');

En train parece que ha mejorado un poco vamos a ver en test

In [ ]:
probas_test = lr.predict_proba(X_test)
preds_test = [1 if probas_test[i][1]>0.30 else 0 for i in range(len(probas_test))]
recall_proba_test = recall_score(y_test, preds_test)
precision_proba_test = precision_score(y_test, preds_test)
f1_score_proba_test = f1_score(y_test, preds_test)
print(f'Recall: {recall_proba_test},\nPrecision: {precision_proba_test},\nf1: {f1_score_proba_test}')

In [ ]:
sns.heatmap(confusion_matrix(y_test, preds_test), annot=True)
plt.title('Confusion Matrix Probas Test');

En test no hay mejora incluso el resultado empeora del f1 empeora pero el recall ha subido, y si nos fijamos en el error Tipo 2, es decir, estamos detectando con bastante acierto los posibles clientes potenciales a dejar la compañia.

### Otros modelos de clasificación

#### Decisión tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
df_le.head()

In [ ]:
X = df_le.drop(['customerID','Churn','ChurnBinary'], axis=1)
y = df_le.Churn

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=.2, stratify=y)

In [ ]:
dt = DecisionTreeClassifier()

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
preds = dt.predict(X_train)

In [ ]:
precision = precision_score(y_train, preds)
recall = recall_score(y_train, preds)
f1 = f1_score(y_train, preds)
print(f'Recall: {recall},\nPrecision: {precision},\nf1: {f1}')

In [ ]:
sns.heatmap(confusion_matrix(y_train, preds), annot=True);

In [ ]:
preds_test = dt.predict(X_test)
precision_test = precision_score(y_test, preds_test)
recall_test = recall_score(y_test, preds_test)
f1_test = f1_score(y_test, preds_test)
print(f'Recall: {recall_test},\nPrecision: {precision_test},\nf1: {f1_test}')

In [ ]:
sns.heatmap(confusion_matrix(y_test, preds_test), annot=True);

In [ ]:
plot_tree(dt)
plt.show();

In [ ]:
feact = dict(zip(df_le.drop(['customerID','Churn','ChurnBinary'], axis=1).columns,dt.feature_importances_))

In [ ]:
plt.figure(figsize=(20,15))
plt.barh(pd.DataFrame(feact, index=[0]).T[0].index, 
         width=pd.DataFrame(feact, index=[0]).T[0]);

In [ ]:
g_feact = []

for k,v in feact.items():
    if v > 0.03:
        g_feact.append(k)
g_feact

In [ ]:
X = df_le[g_feact]
y = df_le.Churn

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=.2, stratify=y)

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
preds = dt.predict(X_train)
precision = precision_score(y_train, preds)
recall = recall_score(y_train, preds)
f1 = f1_score(y_train, preds)
print(f'Recall: {recall},\nPrecision: {precision},\nf1: {f1}')

In [ ]:
sns.heatmap(confusion_matrix(y_train, preds), annot=True);

In [ ]:
preds_test = dt.predict(X_test)
precision_test = precision_score(y_test, preds_test)
recall_test = recall_score(y_test, preds_test)
f1_test = f1_score(y_test, preds_test)
print(f'Recall: {recall_test},\nPrecision: {precision_test},\nf1: {f1_test}')

In [ ]:
sns.heatmap(confusion_matrix(y_test, preds_test), annot=True);

In [ ]:
plt.figure(figsize=(100,70))
plot_tree(dt)
plt.show();

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = df_le.drop(['customerID','Churn','ChurnBinary'], axis=1)
y = df_le.Churn

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=.2, stratify=y)

In [ ]:
rf = RandomForestClassifier()
smote = SMOTE()

In [ ]:
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
preds = rf.predict(X_train)
precision = precision_score(y_train, preds)
recall = recall_score(y_train, preds)
f1 = f1_score(y_train, preds)
print(f'Recall: {recall},\nPrecision: {precision},\nf1: {f1}')

In [ ]:
sns.heatmap(confusion_matrix(y_train, preds), annot=True);

In [ ]:
preds_test = rf.predict(X_test)
precision_test = precision_score(y_test, preds_test)
recall_test = recall_score(y_test, preds_test)
f1_test = f1_score(y_test, preds_test)
print(f'Recall: {recall_test},\nPrecision: {precision_test},\nf1: {f1_test}')

In [ ]:
sns.heatmap(confusion_matrix(y_test, preds_test), annot=True);

In [ ]:
feact = dict(zip(df_le.drop(['customerID','Churn','ChurnBinary'], axis=1).columns,rf.feature_importances_))

In [ ]:
plt.figure(figsize=(20,15))
plt.barh(pd.DataFrame(feact, index=[0]).T[0].index, 
         width=pd.DataFrame(feact, index=[0]).T[0]);

In [ ]:
g_feact = []

for k,v in feact.items():
    if v > 0.03:
        g_feact.append(k)
g_feact

In [ ]:
X = df_le[g_feact]
y = df_le.Churn

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=.2, stratify=y)

In [ ]:
rf = RandomForestClassifier()
smote = SMOTE()

In [ ]:
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
preds = rf.predict(X_train)
precision = precision_score(y_train, preds)
recall = recall_score(y_train, preds)
f1 = f1_score(y_train, preds)
print(f'Recall: {recall},\nPrecision: {precision},\nf1: {f1}')

In [ ]:
sns.heatmap(confusion_matrix(y_train, preds), annot=True);

In [ ]:
preds_test = rf.predict(X_test)
precision_test = precision_score(y_test, preds_test)
recall_test = recall_score(y_test, preds_test)
f1_test = f1_score(y_test, preds_test)
print(f'Recall: {recall_test},\nPrecision: {precision_test},\nf1: {f1_test}')

In [ ]:
sns.heatmap(confusion_matrix(y_test, preds_test), annot=True);

In [ ]:
proba = rf.predict_proba(X_train)
preds = [1 if proba[i][1]>0.35 else 0 for i in range(len(proba))]
precision = precision_score(y_train, preds)
recall = recall_score(y_train, preds)
f1 = f1_score(y_train, preds)
print(f'Recall: {recall},\nPrecision: {precision},\nf1: {f1}')

In [ ]:
sns.heatmap(confusion_matrix(y_train, preds), annot=True);

In [ ]:
proba = rf.predict_proba(X_test)
preds = [1 if proba[i][1]>0.56 else 0 for i in range(len(proba))]
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)
print(f'Recall: {recall},\nPrecision: {precision},\nf1: {f1}')

In [ ]:
sns.heatmap(confusion_matrix(y_test, preds), annot=True);

#### XGBoost

In [ ]:
#!pip install xgboost

In [ ]:
from xgboost import XGBRFClassifier

In [ ]:
X = df_le.drop(['customerID','Churn','ChurnBinary'], axis=1)
y = df_le.Churn

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=.2, stratify=y)

In [ ]:
xgb = XGBRFClassifier()
smote = SMOTE()

In [ ]:
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
xgb.fit(X_train, y_train)

In [ ]:
preds = xgb.predict(X_train)
precision = precision_score(y_train, preds)
recall = recall_score(y_train, preds)
f1 = f1_score(y_train, preds)
print(f'Recall: {recall},\nPrecision: {precision},\nf1: {f1}')

In [ ]:
sns.heatmap(confusion_matrix(y_train, preds), annot=True);

In [ ]:
preds_test = xgb.predict(X_test)
precision_test = precision_score(y_test, preds_test)
recall_test = recall_score(y_test, preds_test)
f1_test = f1_score(y_test, preds_test)
print(f'Recall: {recall_test},\nPrecision: {precision_test},\nf1: {f1_test}')

In [ ]:
sns.heatmap(confusion_matrix(y_test, preds_test), annot=True);

In [ ]:
xgb.feature_importances_

In [ ]:
feact = dict(zip(df_le.drop(['customerID','Churn','ChurnBinary'], axis=1).columns,xgb.feature_importances_))
plt.figure(figsize=(20,15))
plt.barh(pd.DataFrame(feact, index=[0]).T[0].index, 
         width=pd.DataFrame(feact, index=[0]).T[0]);

In [ ]:
g_feact = []

for k,v in feact.items():
    if v > 0.02:
        g_feact.append(k)
g_feact

In [ ]:
X = df_le[g_feact]
y = df_le.Churn

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=.2, stratify=y)

In [ ]:
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
xgb.fit(X_train, y_train)

In [ ]:
preds = xgb.predict(X_train)
precision = precision_score(y_train, preds)
recall = recall_score(y_train, preds)
f1 = f1_score(y_train, preds)
print(f'Recall: {recall},\nPrecision: {precision},\nf1: {f1}')

In [ ]:
sns.heatmap(confusion_matrix(y_train, preds), annot=True);

In [ ]:
preds_test = xgb.predict(X_test)
precision_test = precision_score(y_test, preds_test)
recall_test = recall_score(y_test, preds_test)
f1_test = f1_score(y_test, preds_test)
print(f'Recall: {recall_test},\nPrecision: {precision_test},\nf1: {f1_test}')

In [ ]:
sns.heatmap(confusion_matrix(y_test, preds_test), annot=True);

In [ ]:
proba = xgb.predict_proba(X_test)
preds = [1 if proba[i][1]>0.30 else 0 for i in range(len(proba))]
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)
print(f'Recall: {recall},\nPrecision: {precision},\nf1: {f1}')

In [ ]:
sns.heatmap(confusion_matrix(y_test, preds), annot=True);

Para finalizar podemos corroborar que este tipo de problematica es compleja de solucionar y al final comprobamos que no por usar modelos más complejos vamos a obtener mejores resultados, en este caso el modelo con el que mejores resultados hemos obtenido de forma general ha sido la regresión logística ajustando el umbral de decisión, ya que disminuíamos el número de Falsos Negativos, y por lo tanto la fuga de clientes sin identificar es menor.